## Обзор данных

Начнем исследование с общего обзора данных. Для этого импортируем библиотеку `pandas`, прочитаем с её помощью файл `data.csv` и оценим формат таблицы с помощью метода `head()`. 

In [1]:
import pandas as pd

In [2]:
try: 
    df = pd.read_csv('/datasets/data.csv') #для дальнейшей работы сохраним таблицу в переменной df
except: 
    df = pd.read_csv('C:\\Users\\freak\\Desktop\\Python\\borrowers_liability\\data.csv')

In [3]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получим общую информацию о таблице и проверим её соответствие документации:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Предварительные выводы о данных:** 
- наименования стоблцов присвоены корректно — все буквы в нижнем регистре, отсутствуют лишние пробелы, соблюдены правила "змеиного регистра"; 
- в стоблцах `days_employed` и `total_income` можно заметить более 2000 пропусков, необходимо будет их обработать;
- в стоблце `days_employed` есть отрицательные значения;
- в стоблце `education` данные записаны разным регистром;
- в столбце `purpose` одни и те же цели сформулированы по-разному (сыграть свадьбу, на проведение свадьбы).

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

В ходе предобработки данных необходимо осуществить следующие шаги:
- заполнить пропуски;
- проверить данные на аномалии и устранить их;
- изменить тип данных; 
- обнаружить и удалить дубликаты; 
- провести декомпозицию исходной таблицы;
- провести категоризацию целей и доходов.


### Заполнение пропусков

На данном этапе исследования необходимо изучить и обработать пропуски в стоблцах `days_employed` и `total_income`. 
Для начала найдем общее количество пропусков в столбцах:

In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски в стоблцах `days_employed` и `total_income` составляют 10% от общего количества данных. Однако, данные об общем трудовом стаже и ежемесячном доходе не влияют на исследование. Достаточно заменить их явными обозначениями. Пропуски в столбцах о стаже и доходе являются количественными переменными. Для заполнения пропусков в столбцах будем использовать медианное значение, а не среднее. Среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства. Так как стаж и доходы могут радакально отличаться от клиента к клиенту, медианное значение будет давать нам более объективную картину.

Возможно, произошел сбой при выгрузке данных, либо сотрудник банка (или клиент) не разобрался с правилами заполнения анкеты и внёс не все данные.

Несмотря на то, что масштабная работа с аномалиями будет проводиться в следующем пункте нашего исследования, есть аномалии, которые лучше обработать в рамках текущего пункта. Так для получения более реалистичного представления о медиане, избавимся от отрицательных значений в столбце `days_employed`:

In [6]:
df['days_employed'] = df['days_employed'].apply(abs) #применим к столбцу метод abs, чтобы вернуть модули всех значений столбца

Убедимся в том, что всё прошло успешно: 

In [7]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Отлично, минусы исчезли. 

Найдем медиану, чтобы затем убедиться в правильности заполнения пропусков:

In [8]:
df['days_employed'].median()

2194.220566878695

Заполним пропуски медианой: 

In [9]:
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())

Убедимся в том, что все прошло успешно:

In [10]:
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Для заполнения пропусков в столбце `total_income` будем использовать медианное значение. Чтобы убедиться в правильности заполнения пропусков, сначала найдем медиану: 

In [11]:
df['total_income'].median()

145017.93753253992

Именно это число мы должны увидеть на месте пропусков после заполнения:

In [12]:
df['total_income'] = df['total_income'].fillna(df['total_income'].median())

Проверим, так ли это:

In [13]:
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Действительно, пропуск в строке с индексом `12` получил значение медианы. 

### Проверка данных на аномалии и исправления

Проверим столбец `children`. Данные в этом столбце важны для исследования, поэтому стоит уделить им особое внимание.

Вызовем к столбцу метод `value_counts()`, который возвращает уникальные значения и количество их упоминаний:

In [14]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Налицо две аномалии: отрицательное значение (возможно, это было тире при вводе данных), а также 76 клиентов с 20-ю детьми в семье (возможно, это опечатка). В любом случае, аномалии нужно обработать. 

Уберем отрицательное значение с помощью уже известного нам метода `abs`: 

In [15]:
df['children'] = df['children'].apply(abs)
df['children'].value_counts() #сразу проверим, удалось ли убрать минус

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Осталось решить вопрос с аномально многодетными клиентами. Скорее всего, на месте 20 должно быть число 2. Это больше похоже на правду. Заменим некорректное значение на корректное с помощью метода `replace()`:

In [16]:
df['children'] = df['children'].replace(20, 2)

In [17]:
df['children'].value_counts() #проверим результат работы метода

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Изменение типов данных

Ранее мы заполнили пропуски в двух столбцах медианным значением. Однако, нужно сделать ещё одно преобразование. Следует превратить значение столбцов в целочисленные. Дробная часть не только затрудняет зрительное восприятие таблицы, но и не является значимой для показателя о стаже и доходах. Сначала превратим значения столбца `days_employed` в целые числа для оптимизации датасета:

In [18]:
df['days_employed'] = df['days_employed'].astype('int32') #используем метод astype()
df.head() #сразу проверим результат

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Аналогично столбцу с трудовым стажем, приведем значения столбца с доходами к целочисленным:

In [19]:
df['total_income'] = df['total_income'].astype('int32')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Изменение типов данных прошло успешно. Переходим к работе с дубликатами. 

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


### Удаление дубликатов

Проверим данные на явные дубликаты. Для этого применим метод `duplicated()` и добавим к нему `sum()`, чтобы в итоге получить количество строк с явными дубликатами: 

In [21]:
df.duplicated().sum()

54

Для удаления явных дубликатов будем использовать метод `drop_duplicates()`, а чтобы избежать нарушения порядка индексов, применим метод `reset_index()` с параметром `drop=True`. В итоге получим таблицу без явных дубликатов с корректным порядком индексов.

In [22]:
df = df.drop_duplicates().reset_index(drop=True)

Ещё раз посчитаем явные дубликаты в таблице — убедимся, что полностью от них избавились:

In [23]:
df.duplicated().sum()

0

Теперь обработаем неявные дубликаты. Так, например, в пункте `2.3` можно заметить, что данные в столбце `education` записаны разным регистром. Приведем в порядок этот столбец. 

Ознакомимся с дубликатами с помощью метода `value_counts()`:

In [24]:
df['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Приведем данные об образовании к единому виду — запишем всё в нижнем регистре с помощью метода `str.lower()`: 

In [25]:
df['education'] = df['education'].str.lower()

Проверим, как всё прошло:

In [26]:
df['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Успех! Переходим к обработке следующих столбцов. Проверим на неявные дубликаты столбец `family_status`:

In [27]:
df['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Здесь бросается в глаза только заглавная буква в статусе "Не женат", это не дубликат. Избавимся от неё для сохранения единства стиля:

In [28]:
df['family_status'] = df['family_status'].str.lower()

In [29]:
df['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Ещё один столбец приведен в порядок. Двигаемся дальше. Проверим столбец `gender`:

In [30]:
df['gender'].value_counts()

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

Здесь дубликатов нет. Перейдем к проверке стоблца `income_type`:

In [31]:
df['income_type'].value_counts()

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Столбец с типом занятости в полном порядке. Последний на очереди — столбец с целью `purpose`:

In [32]:
df['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Проблем с регистром тут явно нет. Однако с этим столбцом нужна более масштабная работа — одни и те же цели записаны несколькими разными способами. Проведем категоризацию целей кредита в пункте `2.7` нашего исследования.

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

На данном этапе создадим словари, в которых объясняется значение категорий столбцов с данными об образовании и семейном положении. После создания словарей можно удалить столбцы с полным названием из исходного датафрейма, чтобы его оптимизировать и избавиться от столбцов, которые, по сути, дублируют друг друга. 

Создадим два новых датафрейма со столбцами:
- `education_id` и `education` — в первом;
- `family_status_id` и `family_status` — во втором.


In [33]:
education_log = df[['education_id','education']] #создаем новый датафрейм
education_log.groupby('education_id')['education'].value_counts() #смотрим соответствие уровней образования и их id 

education_id  education          
0             высшее                  5251
1             среднее                15188
2             неоконченное высшее      744
3             начальное                282
4             ученая степень             6
Name: education, dtype: int64

In [34]:
family_log = df[['family_status_id','family_status']] #создаем новый датафрейм 
family_log.groupby('family_status_id')['family_status'].value_counts() #смотрим соответствие статусов и их id  

family_status_id  family_status        
0                 женат / замужем          12344
1                 гражданский брак          4163
2                 вдовец / вдова             959
3                 в разводе                 1195
4                 не женат / не замужем     2810
Name: family_status, dtype: int64

Удалим из исходного датафрейма столбцы `education` и `family_status`, оставив только их идентификаторы: `education_id` и `family_status_id`.

In [35]:
df.head() #для наглядности выведем текущий формат таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [36]:
df_updated = df.drop(columns = ['education', 'family_status'], axis = 1) #проведем удаление методом drop()
df_updated.head() #посмотрим на результат удаления

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Категоризация дохода

На основании диапазонов, указанных ниже, создадим столбец `total_income_category` с категориями:
- 0–30000 — `E`;
- 30001–50000 — `D`;
- 50001–200000 — `C`;
- 200001–1000000 — `B`;
- 1000001 и выше — `A`.

Например, кредитополучателю с доходом 25000 нужно назначить категорию `E`, а клиенту, получающему 235000, — `B`.


Категоризацию дохода будем проводить с помощью функции. Выбор функции для решения задачи обсуловлен тем, что функции более гибкие: в них можно закладывать более сложную логику и добавлять новые требования. Тестировать функции подстановкой разных значений удобнее, чем применять методы Pandas ко всему датафрейму.

На вход функции попадает доход, а возвращает она категорию клиента.

In [37]:
def total_income_category(total_income):
    
        if total_income <= 30000:
            return 'E'
        elif total_income <= 50000:
            return 'D'
        elif total_income <= 200000:
            return 'C'
        elif total_income <= 1000000:
            return 'B'
        elif total_income > 1000000:
            return 'A'


Проверим работу функции: 

In [38]:
total_income_category(25000)

'E'

Написанная функция работает корректно. Осталось создать отдельный столбец с категориями дохода, и в его ячейках записать значения, возвращаемые функцией.
Для этого нужен метод `apply()`: он берёт значения столбца датафрейма и применяет к ним функцию из своего аргумента. Здесь `apply()` следует вызвать для столбца `total_income`, так как в нём содержатся данные, которые функция примет на вход. Аргументом метода станет сама функция `total_income_category`.


In [39]:
df_updated['total_income_category'] = df_updated['total_income'].apply(total_income_category)
df_updated.head() #сразу вызовем первые 10 строк таблицы, чтобы убедиться в том, что метод сработал корректно

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


Код работает правильно, в таблице появился столбец с категорией дохода. 

После успешной категоризации доходов можно удалить из таблицы столбец `total_income`, оставив только категорию:

In [40]:
df_updated = df_updated.drop(columns = ['total_income'], axis = 1) 
df_updated.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,дополнительное образование,B
4,0,340266,53,1,1,F,пенсионер,0,сыграть свадьбу,C


### Категоризация целей кредита

Обратимся к значениям столбца `purpose`, чтобы понять, с чем мы будем работать: 

In [41]:
df_updated['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец `purpose_category`, в который войдут следующие категории:
- операции с автомобилем,
- операции с недвижимостью,
- проведение свадьбы,
- получение образования.


In [42]:
def purpose_category(purpose):
        if 'свадьб' in purpose:
            return 'проведение свадьбы'
        if 'образован' in purpose:
            return 'получение образования'
        if 'автомобил' in purpose:
            return 'операции с автомобилем'
        
        return 'операции с недвижимостью'


In [43]:
purpose_category('покупка жилья') #проверим работоспособность функции

'операции с недвижимостью'

Добавим столбец с категориями целей в нашу таблицу. 

In [44]:
df_updated['purpose_category'] = df_updated['purpose'].apply(purpose_category)
df_updated.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,дополнительное образование,B,получение образования
4,0,340266,53,1,1,F,пенсионер,0,сыграть свадьбу,C,проведение свадьбы


После успешной категоризации целей можно удалить из таблицы столбец `purpose`, оставив только категорию: 

In [45]:
df_updated = df_updated.drop(columns = ['purpose'], axis = 1) 
df_updated.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,B,получение образования
4,0,340266,53,1,1,F,пенсионер,0,C,проведение свадьбы


## Ответы на вопросы

После полной обработки таблицы можно перейти к завершающей части исследования. Ответим на вопросы с помощью данных, получаемых из сводных таблиц. Для создания таких таблиц будем использовать метод `pivot_table()`. 

### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [46]:
pivot_children = df_updated.pivot_table(index='children', columns='debt', values='dob_years', aggfunc='count')
pivot_children

debt,0,1
children,,
0,13044.0,1063.0
1,4411.0,445.0
2,1926.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


Доля должников среди заемщиков с разным кол-вом детей:

In [47]:
pivot_children = df_updated.pivot_table(index=['children'], values='debt', aggfunc='mean') 
pivot_children

,debt
children,
0,0.075353
1,0.091639
2,0.094925
3,0.081818
4,0.097561
5,0.000000


**Вывод 1:** Очевидно, что среди клиентов банка преобладают люди без детей. Также существенную долю клиентов составляют люди с 1-2 детьми. Наибольшая доля должников наблюдается среди клиентов с 2-мя и 4-мя детьми. Среди бездетных клиентов доля должников заметно ниже. А о долгах клиентов с 5-ю детьми и вовсе нет данных. 

### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [48]:
pivot_family_status = df_updated.pivot_table(index='family_status_id', columns='debt', values='days_employed', aggfunc='count')
pivot_family_status.sort_values([0,1], ascending=False) #сортируем по убыванию для комфортного анализа

debt,0,1
family_status_id,,
0,11413,931
1,3775,388
4,2536,274
3,1110,85
2,896,63


Доля должников среди заемщиков с разным семейным положением:

In [49]:
pivot_family_status = df_updated.pivot_table(index=['family_status_id'], values='debt', aggfunc='mean')
pivot_family_status

,debt
family_status_id,
0,0.075421
1,0.093202
2,0.065693
3,0.071130
4,0.097509


**Вывод 2:** Основная часть клиентов банка это люди, состоящие в браке. Другие значительные категории формируют клиенты состоящие в гражданском браке, а также клиенты со статусом не женат/не замужем. Cреди неженатых клиентов, а также клиентов, состоящих в гражданском браке, можно наблюдать большее количество должников.

### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [50]:
pivot_total_income_category = df_updated.pivot_table(index='total_income_category', 
                            columns='debt', values='dob_years', aggfunc='count')
pivot_total_income_category.sort_values([0,1], ascending=False) #сортируем по убыванию для комфортного анализа

debt,0,1
total_income_category,,
C,14673,1360
B,4685,356
D,329,21
A,23,2
E,20,2


Доля должников среди заемщиков с разным уровнем дохода:

In [51]:
pivot_total_income_category = df_updated.pivot_table(index='total_income_category', values='debt', aggfunc='mean')
pivot_total_income_category

,debt
total_income_category,
A,0.080000
B,0.070621
C,0.084825
D,0.060000
E,0.090909


**Вывод 3:** Основная доля заемщиков это люди с доходами в широком диапазоне — 50-200 тысяч (категория С), ещё одна важная категория это клиенты с доходами от 200 тысяч до 1 млн (категория В). При этом доля должников среди категорий С и В радикально не отличается. Также практически не отличается количество людей с самой высокой и самой низкой категориями доходов, их очень мало. Из-за этого создается впечатление, что наименьшее, с точки зрения количества, число должников находится именно в этих категориях (А и Е). Однако, доля должников в категории Е выше, чем в остальных. В то время как в категории D наблюдается наименьшая доля должников. Получается, что с точки зрения дохода, наибольшая доля должников приходится на клиентов с очень низкими (Е) и высокими доходами (С, А). Самые порядочные клиенты находятся в категории D. 

### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [52]:
pivot_purpose_category = df_updated.pivot_table(index='purpose_category', columns='debt', values='dob_years', aggfunc='count')
pivot_purpose_category.sort_values([0,1], ascending=False) #сортируем по убыванию для комфортного анализа

debt,0,1
purpose_category,,
операции с недвижимостью,10032,782
операции с автомобилем,3905,403
получение образования,3644,370
проведение свадьбы,2149,186


Доля должников среди заемщиков с разными целями:

In [53]:
pivot_purpose_category = df_updated.pivot_table(index='purpose_category', values='debt', aggfunc='mean')
pivot_purpose_category

,debt
purpose_category,
операции с автомобилем,0.093547
операции с недвижимостью,0.072314
получение образования,0.092177
проведение свадьбы,0.079657


**Вывод 4:** Самая популярная цель — операции с недвижимостью, что легко объясняется ценами на неё, при этом доля должников тут небольшая. Операции с автомобилем и образование занимают 2-е и 3-е место соответственно. Свадьба не самая популярная категория, хотя количество клиентов здесь тоже не маленькое. Среди желающих решить вопрос с автомобилем и получить образование можно заметить наибольшее количество должников. 

## Итоги исследования:

В ходе исследования мы ответили на 4 вопроса и установили, в каких категориях клиентов больше всего должников. 

1) Среди заемщиков преобладают бездетные, при этом доля должников среди клиентов без детей невысокая, по сравнению с другими категориями. Многодетным клиентам сложнее дается выполнение долговых обязательств. Это можно увидеть в таблице с долей должников. Первый столбец содержит данные о количестве детей, второй — о проценте должников среди заемщиков с указанным количеством детей.

2) Основная масса заемщиков состоит в браке, плюс среди них сравнительно невысокий процент должников. Доля должников среди неженатых и состоящих в гражданском браке заметно выше. Самый низкий процент должников среди овдовевших клиентов, но и обращаются за кредитом такие клиенты реже всех. Ниже преведены результаты исследования: 

3) Для людей с самыми низкими доходами выплата кредита представляет наибольшую сложность. Парадоксально, но и люди с доходами 50-200 тыс, а также более 1 млн тоже испытывают трудности со своевременным возвратом кредита. 


4) Несмотря на популярность операций с недвижимостью, больше всего должников можно заметить среди желающих совершить операции с автомобилем или получить образование. 

Таким образом, можно обозначить портрет максимально ненадежного заемщика: 4 детей, не женат/не замужем, доход 50-200тыс., цель кредита: операции с автомобилем. 

**Важно:** в ходе исследования были обнаружены аномалии в столбце с трудовым стажем. Есть клиенты с трудовым стажем более 900 лет. В случае продолжения исследования стоит уделить больше внимания устранению аномалий в этом столбце. 